In [ ]:
import pandas as pd
import re
import os

# 1. DataFrame에서 특정 패턴과 일치하는 행을 찾는 함수
def find_row_by_pattern(df, pattern):
    for idx, row in df.iterrows():
        if any(re.search(pattern, str(cell), re.IGNORECASE) for cell in row):
            return row
    return None

# 2. 엑셀 파일에서 특정 항목에 해당하는 값을 추출하는 함수
def extract_financial_data(file_path, sheet_names, data_items, date_columns):
    results = []
    try:
        xls = pd.ExcelFile(file_path)
        available_sheets = xls.sheet_names

        for sheet_name in sheet_names:
            if sheet_name in available_sheets:
                df = pd.read_excel(file_path, sheet_name=sheet_name, header=[0, 1])
                df.columns = [f'{col[0]} {col[1]}' if 'Unnamed' not in col[1] else col[0] for col in df.columns]

                # 시트의 표기 금액이 천단위일때
                unit_in_thousands = any('in Thousands' in col for col in df.columns)
                # 시트의 표기 금액이 만단위일때
                unit_in_millions = any('in Millions' in col for col in df.columns)

                # 각 항목에 대해 패턴을 기반으로 데이터를 추출
                items = data_items.get(sheet_name, {})

                for item, pattern in items.items():
                    row = find_row_by_pattern(df, pattern)
                    if row is not None:
                        for date in date_columns:
                            matching_col = [col for col in df.columns if re.search(date, col, re.IGNORECASE)]
                            if matching_col:
                                col_name = matching_col[0]
                                value = row.get(col_name, None)
                                if value is not None:
                                    try:
                                        # 괄호 처리를 통해 음수 값 변환 및 수치 변환
                                        if isinstance(value, str) and '(' in value and ')' in value:
                                            value = -float(value.replace('(', '').replace(')', '').replace(',', ''))
                                        elif isinstance(value, str):
                                            value = float(value.replace(',', ''))
                                        # 단위 조정
                                        if unit_in_thousands:
                                            value = round(value / 1000, 1)  # Thousands to Millions 변환
                                        elif unit_in_millions:
                                            value = round(value, 1)  # Millions 단위 그대로 유지
                                        else:
                                            value = round(value / 1000000, 1)  # Dollars to Millions 변환
                                        results.append({
                                            'Date': date,
                                            'Item': item,
                                            'Value': value
                                        })
                                    except ValueError:
                                        print(f"Skipping non-numeric data in {sheet_name} at {date} for {item}")
                            else:
                                results.append({
                                    'Date': date,
                                    'Item': item,
                                    'Value': None
                                })
                    else:
                        for date in date_columns:
                            results.append({
                                'Date': date,
                                'Item': item,
                                'Value': None
                            })
        return results

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return []

# 기준금리와 주가 데이터를 추가 데이터로 정의
additional_data = {
    'base_rate': {
        'Mar. 31, 2018': 1.625, 'Jun. 30, 2018': 1.875, 'Sep. 30, 2018': 2.125, 'Dec. 31, 2018': 2.375,
        'Mar. 31, 2019': 2.375, 'Jun. 30, 2019': 2.375, 'Sep. 30, 2019': 1.875, 'Dec. 31, 2019': 1.625,
        'Mar. 31, 2020': 0.125, 'Jun. 30, 2020': 0.125, 'Sep. 30, 2020': 0.125, 'Dec. 31, 2020': 0.125,
        'Mar. 31, 2021': 0.125, 'Jun. 30, 2021': 0.125, 'Sep. 30, 2021': 0.125, 'Dec. 31, 2021': 0.125,
        'Mar. 31, 2022': 0.375, 'Jun. 30, 2022': 1.625, 'Sep. 30, 2022': 3.125, 'Dec. 31, 2022': 4.375,
        'Mar. 31, 2023': 4.875, 'Jun. 30, 2023': 5.125, 'Sep. 30, 2023': 5.375, 'Dec. 31, 2023': 5.375,
        'Mar. 31, 2024': 5.375
    }
}

# 파일 경로 및 시트 이름 설정
directory = '/content/'  # 실제 디렉토리 경로로 변경해주세요
sheet_names = ['CONSOLIDATED STATEMENTS OF COND', 'CONSOLIDATED STATEMENTS OF INCO', 'Consolidated Statements Of Cond', 'Consolidated Statements Of Inco', 'Consolidated Statements of Cond', 'Consolidated Statements of Inco']

# 데이터 항목 및 패턴 정의 => 다른 항목을 손수 찾아서 입력
income_data_items = {
    'Interest Income': r'Total interest income',
    'Net Interest Income': r'Net interest income',
    'Non-Interest Income': r'Total non-interest income',
    'Income Before Tax': r'Income before taxes',
    'Tax Expense': r'Income tax expense',
    'Net Income': r'Net income'
}

balance_sheet_items = {
    'Cash and Due from Banks': r'Cash and due from banks',
    'Goodwill': r'Goodwill',
    'Loans': r'Total loans',
    'Net Loans': r'Loans\, net of unearned income\, excluding covered loans|Loans\, net of unearned income', 
    'Total Assets': r'Total assets',
    'Total Liabilities': r'Total liabilities',
    'Total Stockholders Equity': r"Total shareholders' equity",
    'Retained Earnings': r'Retained earnings'
}

data_items = {
    'Consolidated Statements Of Cond': balance_sheet_items,
    'Consolidated Statements Of Inco': income_data_items,
    'Consolidated Statements of Inco': income_data_items,
    'Consolidated Statements of Cond': balance_sheet_items,
    'CONSOLIDATED STATEMENTS OF INCO': income_data_items,
    'CONSOLIDATED STATEMENTS OF COND': balance_sheet_items
    
}

# 날짜 열 정의
date_columns = [
    'Mar. 31, 2018', 'Jun. 30, 2018', 'Sep. 30, 2018', 'Dec. 31, 2018',
    'Mar. 31, 2019', 'Jun. 30, 2019', 'Sep. 30, 2019', 'Dec. 31, 2019',
    'Mar. 31, 2020', 'Jun. 30, 2020', 'Sep. 30, 2020', 'Dec. 31, 2020',
    'Mar. 31, 2021', 'Jun. 30, 2021', 'Sep. 30, 2021', 'Dec. 31, 2021',
    'Mar. 31, 2022', 'Jun. 30, 2022', 'Sep. 30, 2022', 'Dec. 31, 2022',
    'Mar. 31, 2023', 'Jun. 30, 2023', 'Sep. 30, 2023', 'Dec. 31, 2023',
    'Mar. 31, 2024'
]

# 결과를 저장할 리스트 초기화
all_results = []

# 디렉토리 내의 모든 엑셀 파일 처리
for filename in os.listdir(directory):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        bank_id = filename.split('.')[0]
        file_path = os.path
        file_path = os.path.join(directory, filename)
        financial_data = extract_financial_data(file_path, sheet_names, data_items, date_columns)

        if financial_data:
            all_results.extend(financial_data)

# 데이터프레임으로 변환
result_df = pd.DataFrame(all_results)

# Date를 Year와 Quarter로 분리
result_df['Year'] = result_df['Date'].apply(lambda x: x.split(', ')[1])
result_df['Quarter'] = result_df['Date'].apply(lambda x: x.split(', ')[0])

# 피벗 테이블로 재구성
result_pivot = result_df.pivot_table(index=['Year', 'Quarter'], columns='Item', values='Value', aggfunc='first').reset_index()

# 기준금리 추가
base_rate_df = pd.DataFrame(additional_data['base_rate'].items(), columns=['Date', 'Interest_Rate'])
base_rate_df['Year'] = base_rate_df['Date'].apply(lambda x: x.split(', ')[1])
base_rate_df['Quarter'] = base_rate_df['Date'].apply(lambda x: x.split(', ')[0])
base_rate_df.drop(columns='Date', inplace=True)

# 최종 데이터프레임과 병합
final_df = pd.merge(result_pivot, base_rate_df, on=['Year', 'Quarter'], how='left')

# 작업 시 마다 티커명을 바꿔야합니다.
# 기존 데이터프레임에 'Name' 열을 추가하고 이를 맨 앞에 배치
final_df['Name'] = 'WTFC'

# 열 순서를 조정하여 'Name' 열을 맨 앞으로 이동
cols = ['Name'] + [col for col in final_df.columns if col != 'Name']
final_df = final_df[cols]

# 결과 출력
final_df